In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_org = pd.read_csv('data/salary_data_cleaned.csv')
df = pd.read_csv('data/df_13.csv')
df_4 = pd.read_csv('data/df_4.csv')

# Visualizations

### Visualization 1: Geographical Distribution

**Task/Question 1**: How are tech job opportunities distributed across each state in the US in terms of job positions, average salary, and total count of opportunities?

In [4]:
sector_selection = alt.selection_multi(on='click', fields=['job_state'], empty='all')

usa = alt.topo_feature(data.us_10m.url, 'states')

foreground = (
    alt.Chart(usa)
    .mark_geoshape(stroke='#aaa', strokeWidth=0.25)
    .transform_lookup(
        lookup='id',
        from_=alt.LookupData(data=df, key='state_id', fields=['Job Title', 'Average Salary per state', 'count', 'job_state', 'state_id']))
    .encode(
        alt.Color('count:Q', scale=alt.Scale(scheme='viridis'), title='Job Count'),
        alt.Tooltip(['job_state:O', 'Job Title:N', 'Average Salary per state:Q', 'count:Q']),
    )
    .project(type='albersUsa')
    .properties(width=400, height=350)
    .add_selection(sector_selection)
    .transform_filter(
    sector_selection
)
)

background = alt.Chart(usa).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    title = 'Task 1: How are job opportunities distributed across each state in the US?',
    width=400,
    height=350,
).project('albersUsa')

choropleth = background + foreground 
choropleth

alt.LayerChart(...)

**Observation**: Based on the visualization above, we conclude the state of **California (highlighted in yellow) offers the highest volume of job opportunities** in the tech industry across the United Sates. This observation makes reasonable sense, given the Silicon Valley located in California fosters technological developments as a global center.

Please also note that population density is not taken into account in this visualization to calculate number of job opportunities per capita within each state. In other words, the intensity of competition cannot be inferred via this visualization. This is due to limited sample size (e.g. negligible counts in certain states) and a lack of information (e.g. additional data source on the US population at corresponding timestamps) to ensure meaningful data conclusions.

### Visualization 2: Correlation Between Salary and Company Age

**Task/Question 2**: Is there any correlation between the salary offered and the company's age, and if so, what type of correlation? The analysis in this visualization aims to guide these observations with a breakdown by job sectors.

In [5]:
# Function to get the top 3 job titles for each sector
def get_top_titles(df, n=3):
    # Group by 'Sector' and 'Job Title', then count the occurrences
    sector_job_counts = df.groupby(['Sector', 'Job Title']).size().reset_index(name='counts')
    
    # Sort and take the top n titles per sector
    top_titles = sector_job_counts.sort_values(['Sector', 'counts'], ascending=[True, False]) \
                                  .groupby('Sector').head(n)
    return top_titles

# Apply the function to df_2
top_titles = get_top_titles(df)

# Merge with the original DataFrame to filter only the top job titles
df_top_titles = pd.merge(df, top_titles, on=['Sector', 'Job Title'])

# Define a selection that will be shared within each row based on the y-axis
brush = alt.selection_interval(encodings=['y'], resolve='global')
brush_x = alt.selection_interval(encodings = ['x'], resolve = 'global')

# Scatter plot base
scatter_base = alt.Chart().mark_circle().encode(
    x='company_age:Q',
    y=alt.Y('avg_salary:Q', title='Average Salary', scale=alt.Scale(zero=False)),
    color=alt.condition(brush, 'Job Title:N', alt.value('lightgrey')),
    tooltip=['Sector:N', 'company_age:Q', 'avg_salary:Q']
).properties(
    width=150,
    height=60,
).add_params(
    brush,
    brush_x
)

# Line chart for each job title
def line_chart(source, title):
    return alt.Chart(source).mark_line().encode(
        x='company_age:Q',
        y=alt.Y('avg_salary:Q', scale=alt.Scale(zero=False)),
        color=alt.condition(brush, 'Job Title:N', alt.value('lightgrey')),
        tooltip=['Sector:N', 'company_age:Q', 'avg_salary:Q']
    ).transform_filter(
        alt.datum['Job Title'] == title
    ).properties(
        width=50,
        height=60
    ).add_params(
        brush
    )

# Create dropdown for sector selection
sector_dropdown = alt.binding_select(options=df['Sector'].unique())
sector_selection = alt.selection_point(fields=['Sector'], bind=sector_dropdown, name="Select")

# Final chart
final_chart = alt.vconcat(data=df_top_titles)

# Add plots for each sector and top 3 job titles
for sector in df['Sector'].unique():
    sector_df = df_top_titles[df_top_titles['Sector'] == sector]
    top_titles = sector_df['Job Title'].unique()[:3]
    
    # Create scatter and line plots
    scatter_plot = scatter_base.encode().transform_filter(alt.datum.Sector == sector)
    line_plots = [line_chart(sector_df, title) for title in top_titles]
    
    # Combine scatter and line plots horizontally
    combined_plots = alt.hconcat(scatter_plot, *line_plots, spacing=0).resolve_scale(y='shared')
    
    # Add to final chart
    final_chart &= combined_plots

# Add sector selection to the final chart
scatter_matrix = final_chart.properties(title = 'Task 2: Is there any correlation between salary offered and company age?').add_params(sector_selection).transform_filter(sector_selection)

scatter_matrix

alt.VConcatChart(...)

### Visualization 3: Volume of Tech Opportunities Across Sectors

**Task/Question 3**: Which job sectors are the most in demand (i.e. with the highest count of postings)? The following visualization addresses this question and provides descriptive labels of each sector for additional, sector-specific insights on corresponding job positions, company ratings, and average salary.

In [9]:

sector_selection = alt.selection_multi(fields=['Sector'])

# Base chart with transformations
base = (
    alt.Chart(df)
    .transform_aggregate(
        count_="count()", 
        avg_rating="mean(Rating)",
        avg_salary="mean(avg_salary)", 
        groupby=["Sector", "Job Title"]
    )
    .transform_stack(
        stack="count_",
        as_=["stack_count_Sector1", "stack_count_Sector2"],
        offset="normalize",
        sort=[alt.SortField("Sector", "ascending")],
        groupby=[],
    )
    .transform_window(
        x="min(stack_count_Sector1)",
        x2="max(stack_count_Sector2)",
        rank_JobTitle="dense_rank()",
        distinct_JobTitle="distinct(Job Title)",
        groupby=["Sector"],
        frame=[None, None],
        sort=[alt.SortField("Job Title", "ascending")],
    )
    .transform_window(
        rank_Sector="dense_rank()",
        frame=[None, None],
        sort=[alt.SortField("Sector", "ascending")],
    )
    .transform_stack(
        stack="count_",
        groupby=["Sector"],
        as_=["y", "y2"],
        offset="normalize",
        sort=[alt.SortField("Job Title", "ascending")],
    )
    .transform_calculate(
        ny="datum.y",
        ny2="datum.y2",
        nx="datum.x + (datum.rank_Sector - 1) * 0.001/2",
        nx2="datum.x2 + (datum.rank_Sector - 1) * 0.001/2",
        xc="(datum.nx+datum.nx2)/2",
        yc="(datum.ny+datum.ny2)/2",
    )
)

rect = base.mark_rect().encode(
    # x=alt.X("nx:Q").axis(None),
    x = alt.X("nx:Q", axis=alt.Axis(title='Sector')),
    x2="nx2",
    # y="ny:Q",
    y = alt.Y("ny:Q", axis=alt.Axis(title='Job Title')),
    y2="ny2",
    color=alt.Color("Sector:N").legend(None),
    # opacity=alt.Opacity("count_:Q").legend(None),
    tooltip=["Sector:N", "Job Title:N", "avg_rating:Q", 'avg_salary:Q', "count_:Q"]
).add_selection(sector_selection).transform_filter(
    sector_selection
).properties(
    width = 450,
    height = 700,
    title = 'Task 3: Which job positions and sectors are the most in demand?'
)


mosaic = rect

sector_labels = base.mark_text(baseline="middle", align="center").encode(
    text="Sector",
)

mosaic = (
    mosaic
    .resolve_scale(x="shared")
)

mosaic_plot = mosaic.configure_axis(
    domain=False, ticks=False, labels=False, grid=False
).configure_view(stroke="#aaa", strokeWidth = 1.5,
).configure_concat(spacing=10).configure_axis(domain=False, ticks=False, labels=False, grid=False)
mosaic_plot

alt.Chart(...)

**Observation**: Visualization 3 above indicates **Information Technology** to be dominant industry sector in the tech field, followed by Biotech & Pharmaceuticals ranking second and Business Services ranking third. Among all three sectors, the most common job title observed is **Data Scientist**.

### Visualization 4: Demand in Candidates' Skillset

**Task/Question 4**: What skills are most demanded for each job sector?

In [7]:
total_demand = df_4.groupby('Skill')['Jobs'].sum().reset_index()
ranked_skills = total_demand.sort_values(by='Jobs', ascending=False)['Skill'].tolist()

sector_dropdown = alt.selection_multi(
    encodings=['y'], empty='all'
)

heatmap = alt.Chart(df_4).mark_rect().encode(
    x = alt.X('Skill:N', sort=ranked_skills, title='Skill'),
    y = alt.Y('Sector:N', title='Sector'),
    color = alt.Color('Jobs:Q', title='Count of Skills'),
    tooltip = [
        'Sector:N',
        'Skill:N',
        'Jobs:Q',
        'Percentage:Q',
    ]
).add_selection(sector_dropdown).transform_filter(
    sector_dropdown
).properties(
    width = 400,
    height = 800, 
    title='Task 4: What skills are most demanded for a specific industry?'
)
heatmap

alt.Chart(...)

**Observation**: Overall, across all job sectors, the most demanded skill is **Python**. On the other hand, the R language seems have a lower adoption rate/use case within the tech industry based on the Glassdoor dataset. 

# Dashboard

In [10]:
v_chart = alt.vconcat(choropleth, mosaic, heatmap)

dashboard = alt.hconcat(v_chart, final_chart).configure_axis(
    domain=False, ticks=False, labels=False, grid=False
).configure_view(stroke="#aaa", strokeWidth = 1.5,
).configure_concat(spacing=10).configure_axis(domain=False, ticks=False, labels=False, grid=False)

dashboard

alt.HConcatChart(...)